# Лабораторная работа №2
## Изучение библиотек обработки данных. Часть 2.
Часть 2.
Выполните следующие запросы с использованием двух различных библиотек - Pandas и PandaSQL:

-один произвольный запрос на соединение двух наборов данных
-один произвольный запрос на группировку набора данных с использованием функций агрегирования
Сравните время выполнения каждого запроса в Pandas и PandaSQL.

Подключаем библиотеки:

In [74]:
import numpy as np
import pandas as pd
import pandasql as ps
from timeit import default_timer as timer

Датафреймы для двух выборок:

In [75]:
fd = pd.read_csv('C:/jupWork2/data/user_device.csv')
sd = pd.read_csv('C:/jupWork2/data/user_usage.csv')

Часть выборки User_device:

In [76]:
fd.head()

,use_id,user_id,platform,platform_version,device,use_type_id
0,22782,26980,ios,10.2,"iPhone7,2",2
1,22783,29628,android,6.0,Nexus 5,3
2,22784,28473,android,5.1,SM-G903F,1
3,22785,15200,ios,10.2,"iPhone7,2",3
4,22786,28239,android,6.0,ONE E1003,1


Часть выборки User_usege:

In [77]:
sd.head()

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id
0,21.97,4.82,1557.33,22787
1,1710.08,136.88,7267.55,22788
2,1710.08,136.88,7267.55,22789
3,94.46,35.17,519.12,22790
4,71.59,79.26,1557.33,22792


Как видно выше, имеется общее поле use_id. Реализуем соединение двух таблиц по этому полю с использованием PandaSQL:

In [78]:
def example1_pandasql(fd,sd):

    simple_query = '''
        SELECT *
        FROM fd JOIN sd
        WHERE fd.use_id==sd.use_id     
        '''
    return ps.sqldf(simple_query, locals())

In [79]:
example1_pandasql(fd,sd)

,use_id,user_id,platform,platform_version,device,use_type_id,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id
0,22787,12921,android,4.3,GT-I9505,1,21.97,4.82,1557.33,22787
1,22788,28714,android,6.0,SM-G930F,1,1710.08,136.88,7267.55,22788
2,22789,28714,android,6.0,SM-G930F,1,1710.08,136.88,7267.55,22789
3,22790,29592,android,5.1,D2303,1,94.46,35.17,519.12,22790
4,22792,28217,android,5.1,SM-G361F,1,71.59,79.26,1557.33,22792
5,22793,28217,android,5.1,SM-G361F,1,71.59,79.26,1557.33,22793
6,22794,28217,android,5.1,SM-G361F,1,71.59,79.26,519.12,22794
7,22795,28217,android,5.1,SM-G361F,1,71.59,79.26,519.12,22795
8,22799,29643,android,6.0,ONEPLUS A3003,1,30.92,22.77,3114.67,22799
9,22801,10976,android,4.4,GT-I9505,1,69.80,14.70,25955.55,22801


И Pandas. Замерим время выполнения каждого:

In [80]:
t1 = timer()
example1_pandasql(fd,sd)
elapsed1 = timer() - t1
t2 = timer()
fsd = fd.merge(sd)
elapsed2 = timer() - t2
print("PandasSQL: {a} \nPandas: {b} ".format(a=elapsed1,b=elapsed2))  

PandasSQL: 0.042729509363198304 
Pandas: 0.01557342086925928 


Для полученной соединенной таблице попробуем провести агрегирование с группировкой и так же замерим время выполнения:
PandaSQL:

In [81]:
def example2_pandasql(sd):
    aggr_query = '''
        SELECT distinct
            device, avg(monthly_mb) as avg_mb
        FROM sd 
        GROUP BY device
        '''
    return ps.sqldf(aggr_query, locals())

In [82]:
example2_pandasql(fsd)

,device,avg_mb
0,A0001,15573.330000
1,C6603,1557.330000
2,D2303,519.120000
3,D5503,1557.330000
4,D5803,1557.330000
5,D6603,7267.550000
6,E6653,5191.120000
7,EVA-L09,1557.330000
8,F3111,2076.450000
9,GT-I8190N,407.010000


Аналогичная функция на Pandas и подсчет времени выполнения.

In [83]:
t1 = timer()
example2_pandasql(fsd)
elapsed1 = timer() - t1
t2 = timer()
fsd.groupby('device').monthly_mb.mean()
elapsed2 = timer() - t2
print("PandasSQL: {a} \nPandas: {b} ".format(a=elapsed1,b=elapsed2))  

PandasSQL: 0.025251644065065193 
Pandas: 0.004341345082139014 


Как можно видель в обоих случаях Pandas превосходит PandaSQL почти на порядок. Несмотря на то что цифры в данном примере выглядят незначительными, для огромных датасетов параметр времени может стать значительным. Так же видно еще одно преимущество Pandas - размеры кода. То что реализовано на PandaSQL полноцененным функцией-запросом пишется на Pandas в одну строчку. Таким образом за исключением совсем сложных SQL-запросов Pandas будет и быстрее, и короче.